In [ ]:

import ROOT
import os
import numpy as np
ROOT.EnableImplicitMT()

In [ ]:
mc_file_name="far_mc.root"
mc_tuple_name="farTree"
file = ROOT.TFile(mc_file_name)
data_file_name="far_data.root"
data_tuple_name="farTree"
file2= ROOT.TFile(data_file_name)
tree_mc = file.Get(mc_tuple_name)
tree_bg=file2.Get(data_tuple_name)

In [ ]:
rdf_mc=ROOT.RDataFrame(mc_tuple_name,mc_file_name)
rdf_data=ROOT.RDataFrame(data_tuple_name,data_file_name)
print("entries in mc: ", rdf_mc.Count().GetValue())
print("entries in data: ", rdf_data.Count().GetValue())

In [ ]:
# List all branches in the trees
variables = [branch.GetName() for branch in tree_mc.GetListOfBranches()]
print(f"Number of branches in tree_mc: {len(variables)}")
print("Branches in the tree:")
for var in variables:
    print(var)

variables2 = [branch.GetName() for branch in tree_bg.GetListOfBranches()]
print(f"Number of branches in tree_bg: {len(variables2)}")
print("Branches in the tree2:")
for var in variables2:
    print(var)

In [ ]:
# Impose condition for the mother ID
mc = rdf_mc.Filter("mup_MC_MOTHER_ID==25 && mum_MC_MOTHER_ID==25")
bg = rdf_data



In [ ]:
file_name = "randomSample.root"
tree_name = "farTree"

# Generate random sample and data_blind.root if they do not exist or load the existing ones
if os.path.exists(file_name) and os.path.exists("data_blind.root"):
    print(f"Loading existing {file_name} and data_blind.root")
    bg = ROOT.RDataFrame(tree_name, file_name)
    data_blind = ROOT.RDataFrame(tree_name, "data_blind.root")
else:
    print("Generating randomSample.root and data_blind.root")
    rdf2_bg = rdf_data.Define("rnd", "gRandom->Uniform()")
    p = 2 * mc.Count().GetValue() / rdf_data.Count().GetValue()
    
    sample_bg = rdf2_bg.Filter(f"rnd < {p}")
    data_blind = rdf2_bg.Filter(f"rnd >= {p}")
    
    sample_bg.Snapshot(tree_name, file_name, rdf_data.GetColumnNames())
    data_blind.Snapshot(tree_name, "data_blind.root", rdf_data.GetColumnNames())
    
    bg = sample_bg

In [ ]:

#Define new variables
mc = mc.Define("mup_MUONDLL", "mup_MUONLLMU -mup_MUONLLBG") \
       .Define("mup_PID_sum", "(mup_PID_K + mup_PID_P)") \
       .Define("mum_MUONDLL", "mum_MUONLLMU-mum_MUONLLBG") \
       .Define("mum_PID_sum", "(mum_PID_K + mum_PID_P)")\
       .Define("mum_HASRICH", "float(mum_PID_K>10E-06)") \
       .Define("mup_HASRICH", "float(mup_PID_K>10E-06)") \
       .Define("mum_HASRICH_ISNAN","mum_HASRICH!=mum_HASRICH") \
       .Define("mup_HASRICH_ISNAN","mup_HASRICH!=mup_HASRICH")
        

bg = bg.Define("mup_MUONDLL", "mup_MUONLLMU-mup_MUONLLBG") \
       .Define("mup_PID_sum", "(mup_PID_K + mup_PID_P)") \
       .Define("mum_MUONDLL", "mum_MUONLLMU-mum_MUONLLBG") \
       .Define("mum_PID_sum", "(mum_PID_K + mum_PID_P)")\
       .Define("mum_HASRICH", "float(mum_PID_K>10E-06)") \
       .Define("mup_HASRICH", "float(mup_PID_K>10E-06)") \
       .Define("mum_HASRICH_ISNAN","mum_HASRICH!=mum_HASRICH") \
       .Define("mup_HASRICH_ISNAN","mup_HASRICH!=mup_HASRICH")


In [ ]:
# Count of events after the cut (muons) and with the random sample

print("entries in mc after filter: ", mc.Count().GetValue())
print("entries in data: ", bg.Count().GetValue())


In [ ]:
# Define the cuts
cut = (
    "(H_FAR_POCA_Z > 2000 && H_FAR_POCA_Z < 9000) && "
    "(H_DTF_END_VZ > 2000 && H_DTF_END_VZ < 9000) && "
    "(H_DTF_P < 400000) && "
     "(H_DTF_ETA > 3)"
    " && (H_DTF_PV_M>1000)"
)
mc_cut = mc.Filter(cut)
bg_cut = bg.Filter(cut)

# Entries after the different cuts (mother ID and cuts fot the variables) and with the new defined variables
print("entries in mc after cut: ", mc_cut.Count().GetValue())
print("entries in data after cut: ", bg_cut.Count().GetValue())

In [ ]:
# Name for the MC tree name (in a different cell for viability purposes when launching just the final part of the code)
mc_tree_name="mc_tree"

In [ ]:
# Name for the background tree name
bg_tree_name="bg_tree"

# Save the filtered data to ROOT files
mc_cut.Snapshot(mc_tree_name, "mc.root")
bg_cut.Snapshot(bg_tree_name, "bg.root")


In [ ]:
# Load the trees from the saved ROOT files
tchain_mc = ROOT.TChain(mc_tree_name)
tchain_mc.Add("mc.root")

tchain_bg = ROOT.TChain(bg_tree_name)
tchain_bg.Add("bg.root")

print("finished ")

In [ ]:
# Sanity checks for the new variables
print("Eventos con mum_HASRICH_ISNAN en data: ", bg_cut.Filter("mum_HASRICH_ISNAN").Count().GetValue())
print("Eventos con mup_HASRICH_IS NAN en data: ", bg_cut.Filter("mup_HASRICH_ISNAN").Count().GetValue())

print("Eventos con mum_HASRICH_ISNAN in MC ", mc_cut.Filter("mum_HASRICH_ISNAN").Count().GetValue())
print("Eventos con mup_HASRICH_ISNAN in MC: ", mc_cut.Filter("mup_HASRICH_ISNAN").Count().GetValue())

In [ ]:
# Sanity checks for the PID variables

print("Eventos con mup_PID_K = 0 en MC: ", mc_cut.Filter("fabs(mup_PID_K)!=0").Count().GetValue())
print("Eventos con mup_PID_P = 0 en MC: ", mc_cut.Filter("fabs(mup_PID_P)!=0").Count().GetValue())

print("Eventos con mum_PID_K = 0 en MC: ", mc_cut.Filter("fabs(mum_PID_K)!=0").Count().GetValue())
print("Eventos con mum_PID_P = 0 en MC: ", mc_cut.Filter("fabs(mum_PID_P)!=0").Count().GetValue())

print("Eventos con mup_PID_K = 0 en data: ", bg_cut.Filter("fabs(mup_PID_K)!=0").Count().GetValue())
print("Eventos con mup_PID_P = 0 en data: ", bg_cut.Filter("fabs(mup_PID_P)!=0").Count().GetValue())

print("Eventos con mum_PID_K = 0 en data: ", bg_cut.Filter("fabs(mum_PID_K)!=0").Count().GetValue())
print("Eventos con mum_PID_P = 0 en data: ", bg_cut.Filter("fabs(mum_PID_P)!=0").Count().GetValue())

In [ ]:
# Activate TMVA for further analysis
ROOT.TMVA.Tools.Instance() #activa el TMVA

In [ ]:
# Create a new  TFile to store the TMVA output
output_file = ROOT.TFile.Open("TMVA_HGBDT_output_over.root", "RECREATE") #aqui se crea el archivo de salida, se guarda el entrenamiento

In [ ]:
# Create a TMVA factory for the classification task
factory = ROOT.TMVA.Factory("TMVAClassification", output_file,
                            "!V:!Silent:Color:DrawProgressBar:Transformations=I:AnalysisType=Classification") #classification signal vs background

In [ ]:
# Create a TMVA DataLoader to load the data and variables
dataloader= ROOT.TMVA.DataLoader("dataset") #crea el dataloader, que es el que se encarga de cargar los datos y las variables

In [ ]:
# Add the variables to the dataloader
features = [
    "H_DTF_BPVDIRA",
    "H_DTF_BPVFD",
    "H_DTF_BPVIP",
    "H_DTF_BPVVDRHO",
    "H_DTF_ETA",
    "H_FAR_DOCA",
    "H_FAR_VTX_E",
    "mup_FAR_IP",
    "mup_DTF_P",
    "mup_DTF_PT",
    "mup_DTF_ETA",
    "mup_PID_sum",
    "mup_PID_MU",
    "mup_MUONDLL",
    "mup_MUONCATBOOST",
    "mup_MUONCHI2CORR",
    "mup_HASRICH",
    "mum_FAR_IP",
    "mum_DTF_P",
    "mum_DTF_PT",
    "mum_DTF_ETA",
    "mum_MUONCATBOOST",
    "mum_MUONCHI2CORR",
    "mum_MUONDLL",
    "mum_PID_sum",
    "mum_PID_MU",
    "mum_HASRICH"
]
for feat in features:
    dataloader.AddVariable(feat) 


In [ ]:
# Calculate the number of entries in the signal and background trees
nmc = mc.Count().GetValue()
nbg = bg.Count().GetValue()

#Activate just the branches used as features
tchain_mc.SetBranchStatus("*", 0)

for var in features:
    tchain_mc.SetBranchStatus(var, 1)

tchain_bg.SetBranchStatus("*", 0)

for var in features:
    tchain_bg.SetBranchStatus(var, 1)

    
# Add the signal and background trees to the dataloader
dataloader.AddSignalTree(tchain_mc, 1.0) 
dataloader.AddBackgroundTree(tchain_bg, 1.0) 

In [ ]:
# Set some options for the dataloader
dataloader.PrepareTrainingAndTestTree("", "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:SplitSeed=42") 

In [ ]:
# Create the BDTG method with specific hyperparameters
factory.BookMethod(dataloader, ROOT.TMVA.Types.kBDT, "BDTG_1000", "!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2")

In [ ]:
# Create the MLP method with specific hyperparameters
factory.BookMethod(dataloader, ROOT.TMVA.Types.kMLP, "MLP_N+1, N", "H:V:VarTransform=N:NeuronType=tanh:NCycles=180:HiddenLayers=N+1, N:TestRate=5:UseRegulator") #train neural network

In [ ]:
# Train, test, and evaluate all methods
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()

In [ ]:
output_file.Close()

In [ ]:
# Load the trained BDT model, add the variables, book the model and compute the BDT score
ROOT.gInterpreter.Declare(r"""
static TMVA::Reader* reader_bdt = nullptr;

static float H_DTF_BPVDIRA_bdt, H_DTF_BPVFD_bdt, H_DTF_BPVIP_bdt, H_DTF_BPVVDRHO_bdt;
static float H_DTF_ETA_bdt, H_FAR_DOCA_bdt, H_FAR_VTX_E_bdt, mup_FAR_IP_bdt;
static float mup_DTF_P_bdt, mup_DTF_PT_bdt, mup_DTF_ETA_bdt, mup_PID_sum_bdt;
static float mup_PID_MU_bdt, mup_MUONDLL_bdt, mup_MUONCATBOOST_bdt, mup_MUONCHI2CORR_bdt;
static float mup_HASRICH_bdt, mum_FAR_IP_bdt, mum_DTF_P_bdt, mum_DTF_PT_bdt;
static float mum_DTF_ETA_bdt, mum_MUONCATBOOST_bdt, mum_MUONCHI2CORR_bdt;
static float mum_MUONDLL_bdt, mum_PID_sum_bdt, mum_PID_MU_bdt, mum_HASRICH_bdt;

void setupReaderBDT(const char* weightfile) {

    if(!reader_bdt) {
        reader_bdt = new TMVA::Reader("!Color:!Silent");
        reader_bdt->AddVariable("H_DTF_BPVDIRA", &H_DTF_BPVDIRA_bdt);
        reader_bdt->AddVariable("H_DTF_BPVFD", &H_DTF_BPVFD_bdt);
        reader_bdt->AddVariable("H_DTF_BPVIP", &H_DTF_BPVIP_bdt);
        reader_bdt->AddVariable("H_DTF_BPVVDRHO", &H_DTF_BPVVDRHO_bdt);
        reader_bdt->AddVariable("H_DTF_ETA", &H_DTF_ETA_bdt);
        reader_bdt->AddVariable("H_FAR_DOCA", &H_FAR_DOCA_bdt);
        reader_bdt->AddVariable("H_FAR_VTX_E", &H_FAR_VTX_E_bdt);
        reader_bdt->AddVariable("mup_FAR_IP", &mup_FAR_IP_bdt);
        reader_bdt->AddVariable("mup_DTF_P", &mup_DTF_P_bdt);
        reader_bdt->AddVariable("mup_DTF_PT", &mup_DTF_PT_bdt);
        reader_bdt->AddVariable("mup_DTF_ETA", &mup_DTF_ETA_bdt);
        reader_bdt->AddVariable("mup_PID_sum", &mup_PID_sum_bdt);
        reader_bdt->AddVariable("mup_PID_MU", &mup_PID_MU_bdt);
        reader_bdt->AddVariable("mup_MUONDLL", &mup_MUONDLL_bdt);
        reader_bdt->AddVariable("mup_MUONCATBOOST", &mup_MUONCATBOOST_bdt);
        reader_bdt->AddVariable("mup_MUONCHI2CORR", &mup_MUONCHI2CORR_bdt);
        reader_bdt->AddVariable("mup_HASRICH", &mup_HASRICH_bdt);
        reader_bdt->AddVariable("mum_FAR_IP", &mum_FAR_IP_bdt);
        reader_bdt->AddVariable("mum_DTF_P", &mum_DTF_P_bdt);
        reader_bdt->AddVariable("mum_DTF_PT", &mum_DTF_PT_bdt);
        reader_bdt->AddVariable("mum_DTF_ETA", &mum_DTF_ETA_bdt);
        reader_bdt->AddVariable("mum_MUONCATBOOST", &mum_MUONCATBOOST_bdt);
        reader_bdt->AddVariable("mum_MUONCHI2CORR", &mum_MUONCHI2CORR_bdt);
        reader_bdt->AddVariable("mum_MUONDLL", &mum_MUONDLL_bdt);
        reader_bdt->AddVariable("mum_PID_sum", &mum_PID_sum_bdt);
        reader_bdt->AddVariable("mum_PID_MU", &mum_PID_MU_bdt);
        reader_bdt->AddVariable("mum_HASRICH", &mum_HASRICH_bdt);

        reader_bdt->BookMVA("BDTG_1000", weightfile);
    }
}

float computeBDTScore(
    float a, float b, float c, float d,
    float e, float f, float g, float h,
    float i, float j, float k, float l,
    float m, float n, float o, float p,
    float q, float r, float s, float t,
    float u, float v, float w, float x,
    float y, float z, float aa) {
                          
    H_DTF_BPVDIRA_bdt = a;
    H_DTF_BPVFD_bdt = b;
    H_DTF_BPVIP_bdt = c;
    H_DTF_BPVVDRHO_bdt = d;
    H_DTF_ETA_bdt = e;
    H_FAR_DOCA_bdt = f;
    H_FAR_VTX_E_bdt = g;
    mup_FAR_IP_bdt = h;
    mup_DTF_P_bdt = i;
    mup_DTF_PT_bdt = j;
    mup_DTF_ETA_bdt = k;
    mup_PID_sum_bdt = l;
    mup_PID_MU_bdt = m;
    mup_MUONDLL_bdt = n;
    mup_MUONCATBOOST_bdt = o;
    mup_MUONCHI2CORR_bdt = p;
    mup_HASRICH_bdt = q;
    mum_FAR_IP_bdt = r;
    mum_DTF_P_bdt = s;
    mum_DTF_PT_bdt = t;
    mum_DTF_ETA_bdt = u;
    mum_MUONCATBOOST_bdt = v;
    mum_MUONCHI2CORR_bdt = w;
    mum_MUONDLL_bdt = x;
    mum_PID_sum_bdt = y;
    mum_PID_MU_bdt = z;
    mum_HASRICH_bdt = aa;

    return reader_bdt->EvaluateMVA("BDTG_1000");
}
""")


In [ ]:

# Load the trained MLP model, add the variables, book the model and compute the MLP score
ROOT.gInterpreter.Declare(r"""
static thread_local TMVA::Reader* reader_mlp2 = nullptr;

static thread_local float H_DTF_BPVDIRA_mlp2, H_DTF_BPVFD_mlp2, H_DTF_BPVIP_mlp2, H_DTF_BPVVDRHO_mlp2;
static thread_local float H_DTF_ETA_mlp2, H_FAR_DOCA_mlp2, H_FAR_VTX_E_mlp2, mup_FAR_IP_mlp2;
static thread_local float mup_DTF_P_mlp2, mup_DTF_PT_mlp2, mup_DTF_ETA_mlp2, mup_PID_sum_mlp2;
static thread_local float mup_PID_MU_mlp2, mup_MUONDLL_mlp2, mup_MUONCATBOOST_mlp2, mup_MUONCHI2CORR_mlp2;
static thread_local float mup_HASRICH_mlp2, mum_FAR_IP_mlp2, mum_DTF_P_mlp2, mum_DTF_PT_mlp2;
static thread_local float mum_DTF_ETA_mlp2, mum_MUONCATBOOST_mlp2, mum_MUONCHI2CORR_mlp2;
static thread_local float mum_MUONDLL_mlp2, mum_PID_sum_mlp2, mum_PID_MU_mlp2, mum_HASRICH_mlp2;

void setupReaderMLP2(const char* weightfile) {
    if(!reader_mlp2) {reader_mlp2 = new TMVA::Reader("!Color:!Silent");}

    reader_mlp2->AddVariable("H_DTF_BPVDIRA", &H_DTF_BPVDIRA_mlp2);
    reader_mlp2->AddVariable("H_DTF_BPVFD", &H_DTF_BPVFD_mlp2);
    reader_mlp2->AddVariable("H_DTF_BPVIP", &H_DTF_BPVIP_mlp2);
    reader_mlp2->AddVariable("H_DTF_BPVVDRHO", &H_DTF_BPVVDRHO_mlp2);
    reader_mlp2->AddVariable("H_DTF_ETA", &H_DTF_ETA_mlp2);
    reader_mlp2->AddVariable("H_FAR_DOCA", &H_FAR_DOCA_mlp2);
    reader_mlp2->AddVariable("H_FAR_VTX_E", &H_FAR_VTX_E_mlp2);
    reader_mlp2->AddVariable("mup_FAR_IP", &mup_FAR_IP_mlp2);
    reader_mlp2->AddVariable("mup_DTF_P", &mup_DTF_P_mlp2);
    reader_mlp2->AddVariable("mup_DTF_PT", &mup_DTF_PT_mlp2);
    reader_mlp2->AddVariable("mup_DTF_ETA", &mup_DTF_ETA_mlp2);
    reader_mlp2->AddVariable("mup_PID_sum", &mup_PID_sum_mlp2);
    reader_mlp2->AddVariable("mup_PID_MU", &mup_PID_MU_mlp2);
    reader_mlp2->AddVariable("mup_MUONDLL", &mup_MUONDLL_mlp2);
    reader_mlp2->AddVariable("mup_MUONCATBOOST", &mup_MUONCATBOOST_mlp2);
    reader_mlp2->AddVariable("mup_MUONCHI2CORR", &mup_MUONCHI2CORR_mlp2);
    reader_mlp2->AddVariable("mup_HASRICH", &mup_HASRICH_mlp2);
    reader_mlp2->AddVariable("mum_FAR_IP", &mum_FAR_IP_mlp2);
    reader_mlp2->AddVariable("mum_DTF_P", &mum_DTF_P_mlp2);
    reader_mlp2->AddVariable("mum_DTF_PT", &mum_DTF_PT_mlp2);
    reader_mlp2->AddVariable("mum_DTF_ETA", &mum_DTF_ETA_mlp2);
    reader_mlp2->AddVariable("mum_MUONCATBOOST", &mum_MUONCATBOOST_mlp2);
    reader_mlp2->AddVariable("mum_MUONCHI2CORR", &mum_MUONCHI2CORR_mlp2);
    reader_mlp2->AddVariable("mum_MUONDLL", &mum_MUONDLL_mlp2);
    reader_mlp2->AddVariable("mum_PID_sum", &mum_PID_sum_mlp2);
    reader_mlp2->AddVariable("mum_PID_MU", &mum_PID_MU_mlp2);
    reader_mlp2->AddVariable("mum_HASRICH", &mum_HASRICH_mlp2);
                          
    reader_mlp2->BookMVA("MLP_N+1, N", weightfile);
}

float computeMLP2Score(
    float a, float b, float c, float d,
    float e, float f, float g, float h,
    float i, float j, float k, float l,
    float m, float n, float o, float p,
    float q, float r, float s, float t,
    float u, float v, float w, float x,
    float y, float z, float aa) {

    H_DTF_BPVDIRA_mlp2 = a;
    H_DTF_BPVFD_mlp2 = b;
    H_DTF_BPVIP_mlp2 = c;
    H_DTF_BPVVDRHO_mlp2 = d;
    H_DTF_ETA_mlp2 = e;
    H_FAR_DOCA_mlp2 = f;
    H_FAR_VTX_E_mlp2 = g;
    mup_FAR_IP_mlp2 = h;
    mup_DTF_P_mlp2 = i;
    mup_DTF_PT_mlp2 = j;
    mup_DTF_ETA_mlp2 = k;
    mup_PID_sum_mlp2 = l;
    mup_PID_MU_mlp2 = m;
    mup_MUONDLL_mlp2 = n;
    mup_MUONCATBOOST_mlp2 = o;
    mup_MUONCHI2CORR_mlp2 = p;
    mup_HASRICH_mlp2 = q;
    mum_FAR_IP_mlp2 = r;
    mum_DTF_P_mlp2 = s;
    mum_DTF_PT_mlp2 = t;
    mum_DTF_ETA_mlp2 = u;
    mum_MUONCATBOOST_mlp2 = v;
    mum_MUONCHI2CORR_mlp2 = w;
    mum_MUONDLL_mlp2 = x;
    mum_PID_sum_mlp2 = y;
    mum_PID_MU_mlp2 = z;
    mum_HASRICH_mlp2 = aa;

    return reader_mlp2->EvaluateMVA("MLP_N+1, N");
}
""")


In [ ]:
# Set the paths to the trained BDT and MLP weights
bdt_weights_path = "dataset/weights/TMVAClassification_BDTG_1000.weights.xml"
mlp_weights_path = "dataset/weights/TMVAClassification_MLP_N+1, N.weights.xml"


In [ ]:
# Ensure the weights file paths are correct and accessible, and set up the readers
import os

if os.path.exists(bdt_weights_path) and os.path.exists(mlp_weights_path):
	ROOT.setupReaderBDT(bdt_weights_path.encode('utf-8'))  # .encode('utf-8') converts Python string to C-style string
	ROOT.setupReaderMLP2(mlp_weights_path.encode('utf-8'))
else:
	raise FileNotFoundError("One or both weights files are missing. Please check the paths.")
	

In [ ]:
# Shorter name for the data_blind DataFrame
db=data_blind

In [ ]:
# Define new variables for the data_blind DataFrame
db = db.Define("mup_MUONDLL", "mup_MUONLLMU-mup_MUONLLBG") \
       .Define("mup_PID_sum", "(mup_PID_K + mup_PID_P)") \
       .Define("mum_MUONDLL", "mum_MUONLLMU-mum_MUONLLBG") \
       .Define("mum_PID_sum", "(mum_PID_K + mum_PID_P)")\
       .Define("mum_HASRICH", "float(mum_PID_K>10E-06)") \
       .Define("mup_HASRICH", "float(mup_PID_K>10E-06)")


In [ ]:
# Define the cuts for the data_blind DataFrame
db_cut = db.Filter(cut)


# Count the entries in the data_blind DataFrame after the cuts
print("entries in db after cut: ", db_cut.Count().GetValue())

In [ ]:
# Save the filtered data_blind DataFrame to a ROOT file
db_tree_name="db_tree"
db_cut.Snapshot(db_tree_name, "db.root")


In [ ]:
# Charge the necessary variables for the TMVA readers
vars_needed = [
    "H_DTF_BPVDIRA", "H_DTF_BPVFD", "H_DTF_BPVIP", "H_DTF_BPVVDRHO",
    "H_DTF_ETA", "H_FAR_DOCA", "H_FAR_VTX_E", "mup_FAR_IP",
    "mup_DTF_P", "mup_DTF_PT", "mup_DTF_ETA", "mup_PID_sum",
    "mup_PID_MU", "mup_MUONDLL", "mup_MUONCATBOOST", "mup_MUONCHI2CORR",
    "mup_HASRICH", "mum_FAR_IP", "mum_DTF_P", "mum_DTF_PT",
    "mum_DTF_ETA", "mum_MUONCATBOOST", "mum_MUONCHI2CORR",
    "mum_MUONDLL", "mum_PID_sum", "mum_PID_MU", "mum_HASRICH"
]

# Turn off implicit multithreading to avoid issues with TMVA
ROOT.DisableImplicitMT()
# Create a new RDataFrame for the data_blind DataFrame
df = ROOT.RDataFrame("db_tree", "db.root")

In [ ]:
# Define the BDT and MLP scores using the compute functions
df_with_scores  = df.Define("BDT_score", "computeBDTScore(H_DTF_BPVDIRA, H_DTF_BPVFD, H_DTF_BPVIP, H_DTF_BPVVDRHO, H_DTF_ETA, H_FAR_DOCA, H_FAR_VTX_E, mup_FAR_IP, mup_DTF_P, mup_DTF_PT, mup_DTF_ETA, mup_PID_sum,mup_PID_MU, mup_MUONDLL, mup_MUONCATBOOST, mup_MUONCHI2CORR, mup_HASRICH, mum_FAR_IP, mum_DTF_P, mum_DTF_PT, mum_DTF_ETA, mum_MUONCATBOOST, mum_MUONCHI2CORR, mum_MUONDLL, mum_PID_sum, mum_PID_MU, mum_HASRICH)").Define("MLP_score", "computeMLP2Score(H_DTF_BPVDIRA, H_DTF_BPVFD, H_DTF_BPVIP, H_DTF_BPVVDRHO, \
     H_DTF_ETA, H_FAR_DOCA, H_FAR_VTX_E, mup_FAR_IP, \
     mup_DTF_P, mup_DTF_PT, mup_DTF_ETA, mup_PID_sum, \
     mup_PID_MU, mup_MUONDLL, mup_MUONCATBOOST, mup_MUONCHI2CORR, \
     mup_HASRICH, mum_FAR_IP, mum_DTF_P, mum_DTF_PT, \
     mum_DTF_ETA, mum_MUONCATBOOST, mum_MUONCHI2CORR, \
     mum_MUONDLL, mum_PID_sum, mum_PID_MU, mum_HASRICH)")

In [ ]:
# Save more variables for the results in columns and the BDT and MLP scores
all_columns = [str(col) for col in df_with_scores.GetColumnNames()]

columns_to_save = [
    col for col in all_columns
    if (col == "H_DTF_PV_M")
    or (col == "BDT_score")
    or (col == "MLP_score")
    or (col == "H_FAR_VTX_POS_Z")
    or (col == "H_FAR_VTX_POS_Y")
    or (col == "H_FAR_VTX_POS_X")
]

# Save the DataFrame with the selected columns to a new ROOT file
df_with_scores.Snapshot("bg_tree_with_scores", "output_with_bdt_mlp.root", columns_to_save)

print("file was generated: output_with_bdt_mlp.root with selected columns.")


In [ ]:
# Open the file and reload the dataframe to continue without repeating previous steps
df_with_scores = ROOT.RDataFrame("bg_tree_with_scores", "output_with_bdt_mlp.root")


# The other cut (discarded): print("Entries with MLP_score>0.18: ", df_filtered_mlp.Count().GetValue())

# Filter the DataFrame based on the BDT score
df_filtered_bdt = df_with_scores.Filter("BDT_score > 0.02")

# Sanity check for the filtered DataFrame
print("Columns in df_filtered_bdt:", list(df_filtered_bdt.GetColumnNames()))

# Count the number of entries in the filtered DataFrame
print("Entries with BDT_score < 0.02: ", df_filtered_bdt.Count().GetValue())


In [ ]:
# Invariant plot mass:

# Define the variables for the invariant mass plot
variables = {
    "H_DTF_PV_M": (90, 1000, 5300)
}
titles = {
    "H_DTF_PV_M":"H_DTF_PV_M"
}


In [ ]:
# Re-create mc_cut as a new RDataFrame after disabling multithreading
for var, (bins, min_val, max_val) in variables.items():

    

    
    mc_file = ROOT.TFile.Open("mc.root")
    mc_tree = mc_file.Get(mc_tree_name)

    # Add BDT and MLP scores to MC dataframe
    mc_df = ROOT.RDataFrame(mc_tree_name, "mc.root")
    mc_with_scores  = mc_df.Define("BDT_score", "computeBDTScore(H_DTF_BPVDIRA, H_DTF_BPVFD, H_DTF_BPVIP, H_DTF_BPVVDRHO, H_DTF_ETA, H_FAR_DOCA, H_FAR_VTX_E, mup_FAR_IP, mup_DTF_P, mup_DTF_PT, mup_DTF_ETA, mup_PID_sum,mup_PID_MU, mup_MUONDLL, mup_MUONCATBOOST, mup_MUONCHI2CORR, mup_HASRICH, mum_FAR_IP, mum_DTF_P, mum_DTF_PT, mum_DTF_ETA, mum_MUONCATBOOST, mum_MUONCHI2CORR, mum_MUONDLL, mum_PID_sum, mum_PID_MU, mum_HASRICH)").Define("MLP_score", "computeMLP2Score(H_DTF_BPVDIRA, H_DTF_BPVFD, H_DTF_BPVIP, H_DTF_BPVVDRHO, \
     H_DTF_ETA, H_FAR_DOCA, H_FAR_VTX_E, mup_FAR_IP, \
     mup_DTF_P, mup_DTF_PT, mup_DTF_ETA, mup_PID_sum, \
     mup_PID_MU, mup_MUONDLL, mup_MUONCATBOOST, mup_MUONCHI2CORR, \
     mup_HASRICH, mum_FAR_IP, mum_DTF_P, mum_DTF_PT, \
     mum_DTF_ETA, mum_MUONCATBOOST, mum_MUONCHI2CORR, \
     mum_MUONDLL, mum_PID_sum, mum_PID_MU, mum_HASRICH)")

    mc_cut_nom = (
        mc_with_scores
        .Filter("H_DTF_ETA > 3")
        .Filter("BDT_score > 0.02")
    )
    # Filter("MLP_score>0.2999") \
    
    # Create histograms for the invariant mass plot
    mc_hist = mc_cut_nom.Histo1D(ROOT.RDF.TH1DModel(f"h_{var}_mc", f"{var};{var};Events [a.u.]", bins, min_val, max_val), var)
    bg_hist = df_filtered_bdt.Histo1D(ROOT.RDF.TH1DModel(f"h_{var}_bg", f"{var};{var};normalized number of events", bins, min_val, max_val), var)

    
    # Normalise the histograms to the data
    if bg_hist.Integral() > 0:
        mc_hist.Scale(bg_hist.Integral() / mc_hist.Integral())



    # Create a canvas for the plot
    c = ROOT.TCanvas(f"c_{var}", f"c_{var}", 800, 600)
   
    # Set histogram styles
    mc_hist.SetLineColor(ROOT.kBlue)
    bg_hist.SetLineColor(ROOT.kRed)

    # Set axis titles
    mc_hist.GetXaxis().SetTitle("H_DTF_PV_M [MeV]")
    mc_hist.GetYaxis().SetTitle("Events")  

    
    # Draw the histograms
    mc_hist.Draw("HIST")
    bg_hist.Draw("HIST SAME")

    # Set the maximum value for the y-axis and other settings
    max_val = max(mc_hist.GetMaximum(), bg_hist.GetMaximum())
    max_val *= 1.1  # Extra margins for better visibility
    mc_hist.SetMaximum(max_val)
    bg_hist.SetMaximum(max_val) 
    mc_hist.GetYaxis().SetNoExponent(False)
    mc_hist.GetYaxis().SetMaxDigits(3)
    bg_hist.GetYaxis().SetNoExponent(False)
    bg_hist.GetYaxis().SetMaxDigits(3)

    # Add a title to the plot
    plot_title = "Invariant mass distribution of \mu^{+}\mu^{-} (Higgs candidates)" # Usa el nombre de variable si no hay título definido
    mc_hist.SetTitle("")  # Hide the default title
    mc_hist.GetYaxis().SetTitleOffset(1.4)

    # Add a TLatex object for the title
    title_latex = ROOT.TLatex()
    title_latex.SetNDC()
    title_latex.SetTextAlign(22)
    title_latex.SetTextSize(0.044)
    title_latex.SetTextFont(42)
    title_latex.DrawLatex(0.5, 0.94, plot_title)


    # Set y-axis to scientific notation
    mc_hist.SetStats(0)
    bg_hist.SetStats(0)

    # Other settings for the plot
    ROOT.gStyle.SetLegendBorderSize(0)
    ROOT.gStyle.SetLegendFillStyle(0)
    ROOT.gStyle.SetLegendFillColor(0)
    ROOT.gPad.Modified()
    ROOT.gPad.Update()

    # Add a legend and save the plot
    c.Draw()
    legend = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
    legend.AddEntry(mc_hist.GetPtr(), "MC Signal", "l")
    legend.AddEntry(bg_hist.GetPtr(), "Data", "l")
    legend.Draw()
    c.SaveAs(f"invariant_mass/histo_{var}.pdf")

In [ ]:
# Create the histograms for the vertex position variables

# Define the variables for the vertex position histograms
variables = {
    "H_FAR_VTX_POS_Z": (100, -20, 8200),
    "H_FAR_VTX_POS_Y": (100, -1500, 1500),
    "H_FAR_VTX_POS_X": (100, -1500, 1500)
   

}

# Define the titles for the vertex position histograms
titles = {
    "H_FAR_VTX_POS_Y":"H_FAR_VTX_POS_Y [mm]",
    "H_FAR_VTX_POS_X":"H_FAR_VTX_POS_X [mm]",
    "H_FAR_VTX_POS_Z":"H_FAR_VTX_POS_Z [mm]"
    
}

In [ ]:
from itertools import combinations

# Create directories for the histograms
os.makedirs("noise_mc_def", exist_ok=True)
os.makedirs("noise_bg_def", exist_ok=True)

# Iterate over the first three variables  
var_list = list(variables.keys())[:3]  

In [ ]:
# Usar las variables ya definidas en celdas anteriores

# Print available columns for debugging
print("MC columns:", list(mc_cut_nom.GetColumnNames()))
print("BG columns:", list(df_filtered_bdt.GetColumnNames()))

In [ ]:
# Create pairs of variables for the scatter plots
var_parejas = [('H_FAR_VTX_POS_X', y) for y in var_list if y != 'H_FAR_VTX_POS_X']

# Se up options 
ROOT.gROOT.SetBatch(True)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(ROOT.kRainbow) 
ROOT.gStyle.SetTitleOffset(1.4, "Y")

# Create scatter plots for the pairs of variables
for i in range(len(var_parejas)):
    var1, var2 = var_parejas[i]
    nbinsx, xmin, xmax = variables[var1]
    nbinsy, ymin, ymax = variables[var2]

    c1= ROOT.TCanvas(f"c_{var1}_{var2}", "", 800, 600)
    hist_mc = mc_cut_nom.Histo2D(("Distribution of Decay Vertex Positions: Y vs X (MC)", "Distribution of Decay Vertex Positions: Y vs X (MC)", nbinsx, xmin, xmax, nbinsy, ymin, ymax), var1, var2).GetValue()
        
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.04)
    latex.SetTextColor(ROOT.kRed)
    hist_mc.GetXaxis().SetTitle(titles[var1])
    hist_mc.GetYaxis().SetTitle(titles[var2])

    hist_mc.Draw("COLZ")
    c1.Update()
    c1.Draw()

    output_dir = "noise_mc_def"
    output_name = os.path.join(output_dir,f"{var1}_vs_{var2}_scatter.pdf")
    c1.SaveAs(output_name)

    c2= ROOT.TCanvas(f"c_{var1}_{var2}_bg", "", 800, 600)
    hist_bg = df_filtered_bdt.Histo2D(("Distribution of Decay Vertex Positions: Y vs X (BG)", "Distribution of Decay Vertex Positions: Y vs X (data)" , nbinsx, xmin, xmax, nbinsy, ymin, ymax), var1, var2).GetValue()
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.04)
    latex.SetTextColor(ROOT.kRed)
    hist_bg.GetXaxis().SetTitle(titles[var1])
    hist_bg.GetYaxis().SetTitle(titles[var2])
    hist_bg.Draw("COLZ")
    c2.Update()
    c2.Draw()
   
    c2.Update() 
    c2.Draw()
        
    output_dir2= "noise_bg_def"
    output_name2 = os.path.join(output_dir2,f"{var2}_vs_{var1}_scatter.pdf")
    c2.SaveAs(output_name2)

